Useful stuff
---

In [5]:
class AttributeDict(dict):
    """Like dict but with attribute access and setting"""
    __getattr__ = dict.__getitem__
    __setattr__ = dict.__setitem__

In [6]:
import os
import pandas as pd


# Directory containing all train and test csv
dataset_dir = os.path.join(os.curdir, 'datasets', 'comments')

DEFAULT_COLUMNS = ["sentence", "class", "class_idx"]
dataset = AttributeDict(
    train=pd.DataFrame(columns=DEFAULT_COLUMNS),
    test=pd.DataFrame(columns=DEFAULT_COLUMNS)
)

classes = {
        "Donald Trump" : "Donald-Trump-%s.csv",
        "Joe Biden" : "Joe-Biden-%s.csv"
        }


for typ in dataset.keys(): # train, test
    for class_idx, (class_name, class_path) in enumerate(classes.items()): # Donald Trump, Joe Biden
        df = pd.read_csv(os.path.join(dataset_dir, class_path%typ), index_col=0)
        df["class"] =  class_name # Get the name (Donald Trump or Joe Biden)
        df["class_idx"] = class_idx # 0 -> Donald Trump, 1 -> Joe Biden
        df.columns = DEFAULT_COLUMNS # Force columns name
        dataset[typ] = dataset[typ].append(df, ignore_index=True) # Append train from Donald and Joe together (the same for test)

assert dataset.train.shape[1:] == (3,), "dataframe does not contain 3 columns named %s"%DEFAULT_COLUMNS
assert dataset.test.shape[1:] == (3,), "dataframe does not contain 3 columns named %s"%DEFAULT_COLUMNS

print(f'Number of words in our train dataset : {dataset.train.sentence.apply(lambda x:len(x.split(" "))).sum():.3e}')

Number of words in our train dataset : 7.084e+05


Preprocess all sentences
---

- Retrieve entities in the text using spacy `en_core_web_sm` and merged all words making the entity
- Clean the text using custom filters such as : 
    - lower cases
    - strip non alpha numeric values
    - strip punctuation 
    - strip multiple whitespaces
    - finally taking the lemma of the word

In [7]:
from functools import partial
from typing import Sequence, List
from spacy.tokens.doc import Doc
from gensim.models.doc2vec import TaggedDocument, Doc2Vec

from gensim.parsing.preprocessing import preprocess_string
from gensim.parsing.preprocessing import strip_tags       # strip html tags
from gensim.parsing.preprocessing import strip_short
from gensim.parsing.preprocessing import strip_multiple_whitespaces
from gensim.parsing.preprocessing import stem_text
from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.preprocessing import strip_punctuation, strip_non_alphanum
from nltk.stem import WordNetLemmatizer 

lemmatize = WordNetLemmatizer().lemmatize

def tokenize(sentence : str, spacy_model, to_merge_entities:Sequence[str]=["GPE", "LOC", "PERSON"]) -> Doc:
    doc = spacy_model(sentence)

    # Retrieve entities to merge
    ent_to_split = {ent.text:ent.text.split(' ') for ent in doc.ents if ent.label_ in to_merge_entities}

    # Set all entities lemma to the merged name
    for complete_entity, splitted_entity in ent_to_split.items():
        merged_entity = "_".join(splitted_entity)
        sentence.replace(complete_entity, merged_entity)

    CUSTOM_FILTERS = [lambda x: x.lower(),
                      strip_non_alphanum,
                      strip_punctuation,
                      #remove_stopwords,
                      strip_multiple_whitespaces]
    parsed_line = preprocess_string(sentence, CUSTOM_FILTERS)
    parsed_line = [lemmatize(x) for x in parsed_line]

    return parsed_line

def filter_tokens(tokens:Sequence[str], to_avoid:Sequence[str]) -> List[str]:
    returned_tokens = set()
    for token in tokens:
        if len(token)>1 and (token not in to_avoid):
            returned_tokens.add(token)
    return list(returned_tokens)



def sent_preprocess(sentence:str, spacy_model, to_avoid:Sequence[str]) -> List[str]:
    pipe = (partial(tokenize, spacy_model=spacy_model),
            partial(filter_tokens, to_avoid=to_avoid))
    x = sentence
    for f in pipe:
        x = f(x)
    return x


def df_preprocess(df:pd.DataFrame, column:str, inplace:bool=False, spacy_model_name:str="en_core_web_sm") -> pd.DataFrame:
    import spacy
    import string
    from functools import partial

    try:
        nlp = spacy.load(spacy_model_name)
    except OSError as os_error:
        import sys
        import warnings
        import subprocess
        warnings.warn(f"spacy model {spacy_model_name} was not yet installed. Install it now.", ResourceWarning)
        subprocess.check_call([sys.executable, "-m", "spacy", "download", spacy_model_name])
        df_preprocess(df=df, column=column, inplace=inplace, spacy_model_name=spacy_model_name)

    to_avoid = list(string.punctuation+' ') + ['\n', '\t']

    try: # Speedup preprocess with parallelization
        from pandarallel import pandarallel
        pandarallel.initialize(progress_bar=False, verbose=0)
        preprocessed_column = df[column].parallel_apply(partial(sent_preprocess, spacy_model=nlp, to_avoid=to_avoid))
    except AttributeError: # not parallelized
        preprocessed_column = df[column].apply(partial(sent_preprocess, spacy_model=nlp, to_avoid=to_avoid))


    preprocessed_df = df if inplace else df.copy(deep=True)

    preprocessed_df["preprocessed"] = preprocessed_column

    return preprocessed_df

word_set = {
    "Donald Trump" : set(),
    "Joe Biden" : set()
}

def __make_docs(df_row):
    global word_set
    tokens = df_row.preprocessed
    for token in tokens:
        word_set[df_row["class"]].add(token)
    doc = TaggedDocument(words=tokens, tags=[df_row["class"]])
    return doc

def df_make_doc(df:pd.DataFrame, inplace:bool=False) -> pd.DataFrame:

    preprocessed_column = df.apply(__make_docs, axis=1)

    preprocessed_df = df if inplace else df.copy(deep=True)

    preprocessed_df["doc"] = preprocessed_column


    return preprocessed_df


for typ in ("train", "test"):
    df_preprocess(dataset[typ], "sentence", inplace=True) # Preprocess strings
    df_make_doc(dataset[typ], inplace=True) # Build TaggedDocument from tokenized strings

In [8]:
print(dataset.train.tail(1))

print(list(word_set["Joe Biden"])[:5])

                       sentence      class class_idx  \
22569  Go Joe, TAKE TRUMP DOWN!  Joe Biden         1   

                       preprocessed  \
22569  [joe, take, go, down, trump]   

                                               doc  
22569  ([joe, take, go, down, trump], [Joe Biden])  
['48h', 'refill', 'typical', 'poltical', 'converted']


In [9]:
# import re
# import nltk
# from nltk.stem import WordNetLemmatizer
# from pandarallel import pandarallel
# pandarallel.initialize(progress_bar=False)
#
# from gensim.models.doc2vec import Doc2Vec, TaggedDocument
#
# # Used to know which class (Donald, Joe) each word belongs to
# word_set = {
#     "Donald Trump" : set(),
#     "Joe Biden" : set()
# }
#
# def clean_text(text):
#     text = re.sub(r'\|\|\|', r' ', text)
#     text = text.lower()
#     return text
#
# def tokenize_text(text):
#     tokens = []
#     for sent in nltk.sent_tokenize(text):
#         for word in nltk.word_tokenize(sent):
#             if len(word) < 2:
#                 continue
#             tokens.append(word.lower())
#     return tokens
#
# def make_docs(df_row):
#     tokens = tokenize_text(df_row.sentence)
#     for token in tokens:
#         word_set[df_row["class"]].add(token)
#     doc = TaggedDocument(words=tokens, tags=tokens)
#     return doc
#
# # Apply clean text on both train and test dataset
# for typ in ("train", "test"):
#     dataset[typ].sentence = dataset[typ].sentence.parallel_apply(clean_text)
#
# # Apply tokenize and Tagged Documents for both train and test dataset
# for typ in ("train", "test"):
#     dataset[typ]["doc"] = dataset[typ].apply(make_docs, axis=1)
#

Build the Doc2Vec model
---

In [10]:
from tqdm.notebook import tqdm
from multiprocessing import cpu_count

from gensim.models.doc2vec import Doc2Vec

EMBEDDINGS_DIMS = 100
N_WORKERS = 1 # cpu_count() # seems not to parallelize
MIN_COUNT = 2
WINDOW = 2
EPOCHS = 50
MODEL_NAME = f"doc2vec-dims_{EMBEDDINGS_DIMS}-min_count_{MIN_COUNT}-window_{WINDOW}-epochs_{EPOCHS}"

doc2vec_model = Doc2Vec(
    vector_size=EMBEDDINGS_DIMS,
    min_count=MIN_COUNT,
    workers=N_WORKERS,
    epochs=EPOCHS,
    compute_loss=True,
    )

doc2vec_model.build_vocab(dataset.train.doc.values)

In [11]:
from gensim.models.callbacks import CallbackAny2Vec, DiffMetric, ConvergenceMetric, PerplexityMetric
from gensim.test.utils import common_corpus
class PlotLogger(CallbackAny2Vec):
    def __init__(self, model):
        self.epoch = 0
        self.epochs = model.epochs
        self.p = tqdm(total=self.epochs)

    def on_epoch_end(self, model):
        self.epoch += 1
        self.p.update(1)
        if self.epoch == self.epochs: # Because tqdm does not have a write method for pickle dump
            del self.p
        # self.p.set_postfix_str(f"Loss {model.get_latest_training_loss():.3e}")
        
doc2vec_model.train(dataset.train.doc.values,
                   total_examples=doc2vec_model.corpus_count,
                   epochs=doc2vec_model.epochs,
                   callbacks=[PlotLogger(doc2vec_model)]
                   )

In [ ]:
models_path = os.path.join(os.curdir, 'models')
if not os.path.exists(models_path):
    os.mkdir(models_path)

#doc2vec_model.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)
doc2vec_model.save(os.path.join(models_path, MODEL_NAME+".model"))

## Plot the Doc2Vec vectors in 2D using t-SNE

In [16]:
try:
    str.je_lance_une_erreur()
    from tsnecuda import TSNE
    print("Using GPU TSNE")
except:
    print("Using CPU TSNE : ", end='')
    try :
        from MulticoreTSNE import MulticoreTSNE
        TSNE = partial(MulticoreTSNE, n_jobs=cpu_count())
        print("multicore")
    except:
        from sklearn.manifold import TSNE
        print("sklearn")


vec_tsne = TSNE(n_components=2).fit_transform(doc2vec_model.wv.vectors)
vocabulary = list(doc2vec_model.wv.vocab)

Using CPU TSNE : multicore


Word embeddings from Doc2Vec
---

In [17]:
from multiprocessing import Pool
from bokeh.plotting import figure, output_notebook, output_file, show
from bokeh.models import ColumnDataSource
from bokeh.io import curdoc
import numpy as np

def get_class_word(word):
    n = 0
    if word in word_set["Donald Trump"]:
        n += 1
    if word in word_set["Joe Biden"]:
        n += 2
    return n

def get_color_word(class_word:int, color_dict=("#000000", "#FF0000", "#0000FF", "#00FF00")):
#def get_color_word(class_word:int, color_dict=("#000000", "red", "blue", "green")):
    return color_dict[class_word]

def get_class_name_word(class_word, classes=("", "Donald Trump", "Joe Biden", "Biden and Trump")):
    return classes[class_word]

with Pool() as pool:
    class_word = pool.map(get_class_word, vocabulary)
    color_word = pool.map(get_color_word, class_word)
    class_name_word = pool.map(get_class_name_word, class_word)


plot_dict = AttributeDict(
    x = vec_tsne[:,0],
    y = vec_tsne[:,1],
    fill_color = np.array(color_word),
    legend_label = np.array(class_name_word),
    word = np.array(vocabulary),
    class_name_word = np.array(class_name_word)

)
source = ColumnDataSource(plot_dict)

TITLE = "Plot of Doc2Vec vocabulary trained vectors in 2D using t-SNE"
TOOLS = "hover,pan,wheel_zoom,box_zoom,tap,reset,save"
p = figure(tools=TOOLS,
           title=TITLE,
           toolbar_location="above",
           plot_width=800,
           plot_height=800)
p.xaxis.axis_label = "t-SNE variable 1"
p.yaxis.axis_label = "t-SNE variable 2"
output_notebook()

curdoc().theme = "light_minimal"
#output_file("doc2vec.html")

# Show info per circle (pan)
p.hover.tooltips = [
    ("word", "@word"),
    ("class","@class_name_word")
]

# display all circles
color_dict=("#000000", "#FF0000", "#0000FF", "#00FF00")

p.circle(x="x",
         y="y",
         radius=.4,
         selection_color="black",
         fill_alpha=.5,
         line_alpha=.6,
         fill_color="fill_color",
         legend_group='class_name_word',
         source=source)

p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p)

Document embeddings from Doc2Vec
---

In [ ]:
docvecs = doc2vec_model.docvecs
print(docvecs[0])
print(doc2vec_model.infer_vector(dataset.train.doc.values[0].words))

In [ ]:
vec_tsne = TSNE(n_components=2).fit_transform(doc2vec_model.docvecs)

color_word = dataset.train

plot_dict = AttributeDict(
    x = vec_tsne[:,0],
    y = vec_tsne[:,1],
    fill_color = np.array(color_word),
    legend_label = np.array(class_name_word),
    sentence = np.array(dataset.train.sentence.values),
    class_name_word = np.array(dataset.train["class"].values)

)
source = ColumnDataSource(plot_dict)

TITLE = "Plot of Doc2Vec sentence trained vectors in 2D using t-SNE"
TOOLS = "hover,pan,wheel_zoom,box_zoom,tap,reset,save"
p = figure(tools=TOOLS,
           title=TITLE,
           toolbar_location="above",
           plot_width=800,
           plot_height=800)
p.xaxis.axis_label = "t-SNE variable 1"
p.yaxis.axis_label = "t-SNE variable 2"
output_notebook()

curdoc().theme = "light_minimal"
#output_file("doc2vec.html")

# Show info per circle (pan)
p.hover.tooltips = [
    ("sentence", "@sentence"),
    ("class","@class_name_word")
]

# display all circles
color_dict=("#000000", "#FF0000", "#0000FF", "#00FF00")

p.circle(x="x",
         y="y",
         radius=.4,
         selection_color="black",
         fill_alpha=.5,
         line_alpha=.6,
         fill_color="fill_color",
         legend_group='class_name_word',
         source=source)

p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p)

# BERT model

In [ ]:
from sentence_transformers import SentenceTransformer

embedder = SentenceTransformer('roberta-base-nli-stsb-mean-tokens')

encoded_sentences = embedder.encode(dataset.train.sentence.values)

## Get the t-SNE of the BERT model

In [ ]:
vec_tsne_bert = TSNE(n_components=2).fit_transform(encoded_sentences)
vocabulary = dataset.train.sentence.values

In [ ]:
plot_dict = AttributeDict(
    x = vec_tsne_bert[:,0],
    y = vec_tsne_bert[:,1],
    sentence = np.array(vocabulary),
    class_name_word = np.array(class_name_word)

)
source = ColumnDataSource(plot_dict)

TITLE = "Plot of BERT trained vectors in 2D using t-SNE"
TOOLS = "hover,pan,wheel_zoom,box_zoom,tap,reset,save"
p = figure(tools=TOOLS,
           title=TITLE,
           toolbar_location="above",
           plot_width=800,
           plot_height=800)
p.xaxis.axis_label = "t-SNE variable 1"
p.yaxis.axis_label = "t-SNE variable 2"

curdoc().theme = "light_minimal"
output_notebook()
#output_file("doc2vec.html")

# Show info per circle (pan)
p.hover.tooltips = [
    ("sentence", "@sentence"),
]

# display all circles
color_dict=("#000000", "#FF0000", "#0000FF", "#00FF00")

p.circle(x="x",
         y="y",
         radius=.4,
         selection_color="black",
         fill_alpha=.5,
         line_alpha=.6,
         source=source)

p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p)

In [ ]:
plot_dict = AttributeDict(
    x = vec_tsne_bert[:,0],
    y = vec_tsne_bert[:,1],
    sentence = np.array(vocabulary),
    class_name_word = np.array(class_name_word)

)
source = ColumnDataSource(plot_dict)

TITLE = "Plot of BERT trained vectors in 2D using t-SNE"
TOOLS = "hover,pan,wheel_zoom,box_zoom,tap,reset,save"
p = figure(tools=TOOLS,
           title=TITLE,
           toolbar_location="above",
           plot_width=800,
           plot_height=800)
p.xaxis.axis_label = "t-SNE variable 1"
p.yaxis.axis_label = "t-SNE variable 2"

curdoc().theme = "light_minimal"
output_notebook()
#output_file("doc2vec.html")

# Show info per circle (pan)
p.hover.tooltips = [
    ("sentence", "@sentence"),
]

# display all circles
color_dict=("#000000", "#FF0000", "#0000FF", "#00FF00")

p.circle(x="x",
         y="y",
         radius=.4,
         selection_color="black",
         fill_alpha=.5,
         line_alpha=.6,
         source=source)

p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p)

In [ ]:
plot_dict = AttributeDict(
    x = vec_tsne_bert[:,0],
    y = vec_tsne_bert[:,1],
    sentence = np.array(vocabulary),
    class_name_word = np.array(class_name_word)

)
source = ColumnDataSource(plot_dict)

TITLE = "Plot of BERT trained vectors in 2D using t-SNE"
TOOLS = "hover,pan,wheel_zoom,box_zoom,tap,reset,save"
p = figure(tools=TOOLS,
           title=TITLE,
           toolbar_location="above",
           plot_width=800,
           plot_height=800)
p.xaxis.axis_label = "t-SNE variable 1"
p.yaxis.axis_label = "t-SNE variable 2"

curdoc().theme = "light_minimal"
output_notebook()
#output_file("doc2vec.html")

# Show info per circle (pan)
p.hover.tooltips = [
    ("sentence", "@sentence"),
]

# display all circles
color_dict=("#000000", "#FF0000", "#0000FF", "#00FF00")

p.circle(x="x",
         y="y",
         radius=.4,
         selection_color="black",
         fill_alpha=.5,
         line_alpha=.6,
         source=source)

p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p)

plot_dict = AttributeDict(
    x = vec_tsne_bert[:,0],
    y = vec_tsne_bert[:,1],
    sentence = np.array(vocabulary),
    class_name_word = np.array(class_name_word)

)
source = ColumnDataSource(plot_dict)

TITLE = "Plot of BERT trained vectors in 2D using t-SNE"
TOOLS = "hover,pan,wheel_zoom,box_zoom,tap,reset,save"
p = figure(tools=TOOLS,
           title=TITLE,
           toolbar_location="above",
           plot_width=800,
           plot_height=800)
p.xaxis.axis_label = "t-SNE variable 1"
p.yaxis.axis_label = "t-SNE variable 2"

curdoc().theme = "light_minimal"
output_notebook()
#output_file("doc2vec.html")

# Show info per circle (pan)
p.hover.tooltips = [
    ("sentence", "@sentence"),
]

# display all circles
color_dict=("#000000", "#FF0000", "#0000FF", "#00FF00")

p.circle(x="x",
         y="y",
         radius=.4,
         selection_color="black",
         fill_alpha=.5,
         line_alpha=.6,
         source=source)

p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p)

In [ ]:
docvecs = doc2vec_model.docvecs
print(docvecs[0])
print(doc2vec_model.infer_vector(dataset.train.doc.values[0].words))

In [ ]:
vec_tsne = TSNE(n_components=2).fit_transform(doc2vec_model.docvecs)

color_word = dataset.train

plot_dict = AttributeDict(
    x = vec_tsne[:,0],
    y = vec_tsne[:,1],
    fill_color = np.array(color_word),
    legend_label = np.array(class_name_word),
    sentence = np.array(dataset.train.sentence.values),
    class_name_word = np.array(dataset.train["class"].values)

)
source = ColumnDataSource(plot_dict)

TITLE = "Plot of Doc2Vec sentence trained vectors in 2D using t-SNE"
TOOLS = "hover,pan,wheel_zoom,box_zoom,tap,reset,save"
p = figure(tools=TOOLS,
           title=TITLE,
           toolbar_location="above",
           plot_width=800,
           plot_height=800)
p.xaxis.axis_label = "t-SNE variable 1"
p.yaxis.axis_label = "t-SNE variable 2"
output_notebook()

curdoc().theme = "light_minimal"
#output_file("doc2vec.html")

# Show info per circle (pan)
p.hover.tooltips = [
    ("sentence", "@sentence"),
    ("class","@class_name_word")
]

# display all circles
color_dict=("#000000", "#FF0000", "#0000FF", "#00FF00")

p.circle(x="x",
         y="y",
         radius=.4,
         selection_color="black",
         fill_alpha=.5,
         line_alpha=.6,
         fill_color="fill_color",
         legend_group='class_name_word',
         source=source)

p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p)

# BERT model

In [ ]:
from sentence_transformers import SentenceTransformer

embedder = SentenceTransformer('roberta-base-nli-stsb-mean-tokens')

encoded_sentences = embedder.encode(dataset.train.sentence.values)

## Get the t-SNE of the BERT model

In [ ]:
vec_tsne_bert = TSNE(n_components=2).fit_transform(encoded_sentences)
vocabulary = dataset.train.sentence.values

In [ ]:
plot_dict = AttributeDict(
    x = vec_tsne_bert[:,0],
    y = vec_tsne_bert[:,1],
    sentence = np.array(vocabulary),
    class_name_word = np.array(class_name_word)

)
source = ColumnDataSource(plot_dict)

TITLE = "Plot of BERT trained vectors in 2D using t-SNE"
TOOLS = "hover,pan,wheel_zoom,box_zoom,tap,reset,save"
p = figure(tools=TOOLS,
           title=TITLE,
           toolbar_location="above",
           plot_width=800,
           plot_height=800)
p.xaxis.axis_label = "t-SNE variable 1"
p.yaxis.axis_label = "t-SNE variable 2"

curdoc().theme = "light_minimal"
output_notebook()
#output_file("doc2vec.html")

# Show info per circle (pan)
p.hover.tooltips = [
    ("sentence", "@sentence"),
]

# display all circles
color_dict=("#000000", "#FF0000", "#0000FF", "#00FF00")

p.circle(x="x",
         y="y",
         radius=.4,
         selection_color="black",
         fill_alpha=.5,
         line_alpha=.6,
         source=source)

p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p)

In [ ]:
plot_dict = AttributeDict(
    x = vec_tsne_bert[:,0],
    y = vec_tsne_bert[:,1],
    sentence = np.array(vocabulary),
    class_name_word = np.array(class_name_word)

)
source = ColumnDataSource(plot_dict)

TITLE = "Plot of BERT trained vectors in 2D using t-SNE"
TOOLS = "hover,pan,wheel_zoom,box_zoom,tap,reset,save"
p = figure(tools=TOOLS,
           title=TITLE,
           toolbar_location="above",
           plot_width=800,
           plot_height=800)
p.xaxis.axis_label = "t-SNE variable 1"
p.yaxis.axis_label = "t-SNE variable 2"

curdoc().theme = "light_minimal"
output_notebook()
#output_file("doc2vec.html")

# Show info per circle (pan)
p.hover.tooltips = [
    ("sentence", "@sentence"),
]

# display all circles
color_dict=("#000000", "#FF0000", "#0000FF", "#00FF00")

p.circle(x="x",
         y="y",
         radius=.4,
         selection_color="black",
         fill_alpha=.5,
         line_alpha=.6,
         source=source)

p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p)

In [ ]:
plot_dict = AttributeDict(
    x = vec_tsne_bert[:,0],
    y = vec_tsne_bert[:,1],
    sentence = np.array(vocabulary),
    class_name_word = np.array(class_name_word)

)
source = ColumnDataSource(plot_dict)

TITLE = "Plot of BERT trained vectors in 2D using t-SNE"
TOOLS = "hover,pan,wheel_zoom,box_zoom,tap,reset,save"
p = figure(tools=TOOLS,
           title=TITLE,
           toolbar_location="above",
           plot_width=800,
           plot_height=800)
p.xaxis.axis_label = "t-SNE variable 1"
p.yaxis.axis_label = "t-SNE variable 2"

curdoc().theme = "light_minimal"
output_notebook()
#output_file("doc2vec.html")

# Show info per circle (pan)
p.hover.tooltips = [
    ("sentence", "@sentence"),
]

# display all circles
color_dict=("#000000", "#FF0000", "#0000FF", "#00FF00")

p.circle(x="x",
         y="y",
         radius=.4,
         selection_color="black",
         fill_alpha=.5,
         line_alpha=.6,
         source=source)

p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p)

In [ ]:
plot_dict = AttributeDict(
    x = vec_tsne_bert[:,0],
    y = vec_tsne_bert[:,1],
    sentence = np.array(vocabulary),
    class_name_word = np.array(class_name_word)

)
source = ColumnDataSource(plot_dict)

TITLE = "Plot of BERT trained vectors in 2D using t-SNE"
TOOLS = "hover,pan,wheel_zoom,box_zoom,tap,reset,save"
p = figure(tools=TOOLS,
           title=TITLE,
           toolbar_location="above",
           plot_width=800,
           plot_height=800)
p.xaxis.axis_label = "t-SNE variable 1"
p.yaxis.axis_label = "t-SNE variable 2"

curdoc().theme = "light_minimal"
output_notebook()
#output_file("doc2vec.html")

# Show info per circle (pan)
p.hover.tooltips = [
    ("sentence", "@sentence"),
]

# display all circles
color_dict=("#000000", "#FF0000", "#0000FF", "#00FF00")

p.circle(x="x",
         y="y",
         radius=.4,
         selection_color="black",
         fill_alpha=.5,
         line_alpha=.6,
         source=source)

p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p)

In [ ]:
docvecs = doc2vec_model.docvecs
print(docvecs[0])
print(doc2vec_model.infer_vector(dataset.train.doc.values[0].words))

In [ ]:
vec_tsne = TSNE(n_components=2).fit_transform(doc2vec_model.docvecs)

color_word = dataset.train

plot_dict = AttributeDict(
    x = vec_tsne[:,0],
    y = vec_tsne[:,1],
    fill_color = np.array(color_word),
    legend_label = np.array(class_name_word),
    sentence = np.array(dataset.train.sentence.values),
    class_name_word = np.array(dataset.train["class"].values)

)
source = ColumnDataSource(plot_dict)

TITLE = "Plot of Doc2Vec sentence trained vectors in 2D using t-SNE"
TOOLS = "hover,pan,wheel_zoom,box_zoom,tap,reset,save"
p = figure(tools=TOOLS,
           title=TITLE,
           toolbar_location="above",
           plot_width=800,
           plot_height=800)
p.xaxis.axis_label = "t-SNE variable 1"
p.yaxis.axis_label = "t-SNE variable 2"
output_notebook()

curdoc().theme = "light_minimal"
#output_file("doc2vec.html")

# Show info per circle (pan)
p.hover.tooltips = [
    ("sentence", "@sentence"),
    ("class","@class_name_word")
]

# display all circles
color_dict=("#000000", "#FF0000", "#0000FF", "#00FF00")

p.circle(x="x",
         y="y",
         radius=.4,
         selection_color="black",
         fill_alpha=.5,
         line_alpha=.6,
         fill_color="fill_color",
         legend_group='class_name_word',
         source=source)

p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p)

# BERT model

In [ ]:
from sentence_transformers import SentenceTransformer

embedder = SentenceTransformer('roberta-base-nli-stsb-mean-tokens')

encoded_sentences = embedder.encode(dataset.train.sentence.values)

## Get the t-SNE of the BERT model

In [ ]:
vec_tsne_bert = TSNE(n_components=2).fit_transform(encoded_sentences)
vocabulary = dataset.train.sentence.values

In [ ]:
plot_dict = AttributeDict(
    x = vec_tsne_bert[:,0],
    y = vec_tsne_bert[:,1],
    sentence = np.array(vocabulary),
    class_name_word = np.array(class_name_word)

)
source = ColumnDataSource(plot_dict)

TITLE = "Plot of BERT trained vectors in 2D using t-SNE"
TOOLS = "hover,pan,wheel_zoom,box_zoom,tap,reset,save"
p = figure(tools=TOOLS,
           title=TITLE,
           toolbar_location="above",
           plot_width=800,
           plot_height=800)
p.xaxis.axis_label = "t-SNE variable 1"
p.yaxis.axis_label = "t-SNE variable 2"

curdoc().theme = "light_minimal"
output_notebook()
#output_file("doc2vec.html")

# Show info per circle (pan)
p.hover.tooltips = [
    ("sentence", "@sentence"),
]

# display all circles
color_dict=("#000000", "#FF0000", "#0000FF", "#00FF00")

p.circle(x="x",
         y="y",
         radius=.4,
         selection_color="black",
         fill_alpha=.5,
         line_alpha=.6,
         source=source)

p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p)

In [ ]:
plot_dict = AttributeDict(
    x = vec_tsne_bert[:,0],
    y = vec_tsne_bert[:,1],
    sentence = np.array(vocabulary),
    class_name_word = np.array(class_name_word)

)
source = ColumnDataSource(plot_dict)

TITLE = "Plot of BERT trained vectors in 2D using t-SNE"
TOOLS = "hover,pan,wheel_zoom,box_zoom,tap,reset,save"
p = figure(tools=TOOLS,
           title=TITLE,
           toolbar_location="above",
           plot_width=800,
           plot_height=800)
p.xaxis.axis_label = "t-SNE variable 1"
p.yaxis.axis_label = "t-SNE variable 2"

curdoc().theme = "light_minimal"
output_notebook()
#output_file("doc2vec.html")

# Show info per circle (pan)
p.hover.tooltips = [
    ("sentence", "@sentence"),
]

# display all circles
color_dict=("#000000", "#FF0000", "#0000FF", "#00FF00")

p.circle(x="x",
         y="y",
         radius=.4,
         selection_color="black",
         fill_alpha=.5,
         line_alpha=.6,
         source=source)

p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p)

In [ ]:
plot_dict = AttributeDict(
    x = vec_tsne_bert[:,0],
    y = vec_tsne_bert[:,1],
    sentence = np.array(vocabulary),
    class_name_word = np.array(class_name_word)

)
source = ColumnDataSource(plot_dict)

TITLE = "Plot of BERT trained vectors in 2D using t-SNE"
TOOLS = "hover,pan,wheel_zoom,box_zoom,tap,reset,save"
p = figure(tools=TOOLS,
           title=TITLE,
           toolbar_location="above",
           plot_width=800,
           plot_height=800)
p.xaxis.axis_label = "t-SNE variable 1"
p.yaxis.axis_label = "t-SNE variable 2"

curdoc().theme = "light_minimal"
output_notebook()
#output_file("doc2vec.html")

# Show info per circle (pan)
p.hover.tooltips = [
    ("sentence", "@sentence"),
]

# display all circles
color_dict=("#000000", "#FF0000", "#0000FF", "#00FF00")

p.circle(x="x",
         y="y",
         radius=.4,
         selection_color="black",
         fill_alpha=.5,
         line_alpha=.6,
         source=source)

p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p)

plot_dict = AttributeDict(
    x = vec_tsne_bert[:,0],
    y = vec_tsne_bert[:,1],
    sentence = np.array(vocabulary),
    class_name_word = np.array(class_name_word)

)
source = ColumnDataSource(plot_dict)

TITLE = "Plot of BERT trained vectors in 2D using t-SNE"
TOOLS = "hover,pan,wheel_zoom,box_zoom,tap,reset,save"
p = figure(tools=TOOLS,
           title=TITLE,
           toolbar_location="above",
           plot_width=800,
           plot_height=800)
p.xaxis.axis_label = "t-SNE variable 1"
p.yaxis.axis_label = "t-SNE variable 2"

curdoc().theme = "light_minimal"
output_notebook()
#output_file("doc2vec.html")

# Show info per circle (pan)
p.hover.tooltips = [
    ("sentence", "@sentence"),
]

# display all circles
color_dict=("#000000", "#FF0000", "#0000FF", "#00FF00")

p.circle(x="x",
         y="y",
         radius=.4,
         selection_color="black",
         fill_alpha=.5,
         line_alpha=.6,
         source=source)

p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p)

In [ ]:
docvecs = doc2vec_model.docvecs
print(docvecs[0])
print(doc2vec_model.infer_vector(dataset.train.doc.values[0].words))

In [ ]:
vec_tsne = TSNE(n_components=2).fit_transform(doc2vec_model.docvecs)

color_word = dataset.train

plot_dict = AttributeDict(
    x = vec_tsne[:,0],
    y = vec_tsne[:,1],
    fill_color = np.array(color_word),
    legend_label = np.array(class_name_word),
    sentence = np.array(dataset.train.sentence.values),
    class_name_word = np.array(dataset.train["class"].values)

)
source = ColumnDataSource(plot_dict)

TITLE = "Plot of Doc2Vec sentence trained vectors in 2D using t-SNE"
TOOLS = "hover,pan,wheel_zoom,box_zoom,tap,reset,save"
p = figure(tools=TOOLS,
           title=TITLE,
           toolbar_location="above",
           plot_width=800,
           plot_height=800)
p.xaxis.axis_label = "t-SNE variable 1"
p.yaxis.axis_label = "t-SNE variable 2"
output_notebook()

curdoc().theme = "light_minimal"
#output_file("doc2vec.html")

# Show info per circle (pan)
p.hover.tooltips = [
    ("sentence", "@sentence"),
    ("class","@class_name_word")
]

# display all circles
color_dict=("#000000", "#FF0000", "#0000FF", "#00FF00")

p.circle(x="x",
         y="y",
         radius=.4,
         selection_color="black",
         fill_alpha=.5,
         line_alpha=.6,
         fill_color="fill_color",
         legend_group='class_name_word',
         source=source)

p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p)

# BERT model

In [ ]:
from sentence_transformers import SentenceTransformer

embedder = SentenceTransformer('roberta-base-nli-stsb-mean-tokens')

encoded_sentences = embedder.encode(dataset.train.sentence.values)

## Get the t-SNE of the BERT model

In [ ]:
vec_tsne_bert = TSNE(n_components=2).fit_transform(encoded_sentences)
vocabulary = dataset.train.sentence.values

In [ ]:
plot_dict = AttributeDict(
    x = vec_tsne_bert[:,0],
    y = vec_tsne_bert[:,1],
    sentence = np.array(vocabulary),
    class_name_word = np.array(class_name_word)

)
source = ColumnDataSource(plot_dict)

TITLE = "Plot of BERT trained vectors in 2D using t-SNE"
TOOLS = "hover,pan,wheel_zoom,box_zoom,tap,reset,save"
p = figure(tools=TOOLS,
           title=TITLE,
           toolbar_location="above",
           plot_width=800,
           plot_height=800)
p.xaxis.axis_label = "t-SNE variable 1"
p.yaxis.axis_label = "t-SNE variable 2"

curdoc().theme = "light_minimal"
output_notebook()
#output_file("doc2vec.html")

# Show info per circle (pan)
p.hover.tooltips = [
    ("sentence", "@sentence"),
]

# display all circles
color_dict=("#000000", "#FF0000", "#0000FF", "#00FF00")

p.circle(x="x",
         y="y",
         radius=.4,
         selection_color="black",
         fill_alpha=.5,
         line_alpha=.6,
         source=source)

p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p)

In [ ]:
plot_dict = AttributeDict(
    x = vec_tsne_bert[:,0],
    y = vec_tsne_bert[:,1],
    sentence = np.array(vocabulary),
    class_name_word = np.array(class_name_word)

)
source = ColumnDataSource(plot_dict)

TITLE = "Plot of BERT trained vectors in 2D using t-SNE"
TOOLS = "hover,pan,wheel_zoom,box_zoom,tap,reset,save"
p = figure(tools=TOOLS,
           title=TITLE,
           toolbar_location="above",
           plot_width=800,
           plot_height=800)
p.xaxis.axis_label = "t-SNE variable 1"
p.yaxis.axis_label = "t-SNE variable 2"

curdoc().theme = "light_minimal"
output_notebook()
#output_file("doc2vec.html")

# Show info per circle (pan)
p.hover.tooltips = [
    ("sentence", "@sentence"),
]

# display all circles
color_dict=("#000000", "#FF0000", "#0000FF", "#00FF00")

p.circle(x="x",
         y="y",
         radius=.4,
         selection_color="black",
         fill_alpha=.5,
         line_alpha=.6,
         source=source)

p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p)

In [ ]:
plot_dict = AttributeDict(
    x = vec_tsne_bert[:,0],
    y = vec_tsne_bert[:,1],
    sentence = np.array(vocabulary),
    class_name_word = np.array(class_name_word)

)
source = ColumnDataSource(plot_dict)

TITLE = "Plot of BERT trained vectors in 2D using t-SNE"
TOOLS = "hover,pan,wheel_zoom,box_zoom,tap,reset,save"
p = figure(tools=TOOLS,
           title=TITLE,
           toolbar_location="above",
           plot_width=800,
           plot_height=800)
p.xaxis.axis_label = "t-SNE variable 1"
p.yaxis.axis_label = "t-SNE variable 2"

curdoc().theme = "light_minimal"
output_notebook()
#output_file("doc2vec.html")

# Show info per circle (pan)
p.hover.tooltips = [
    ("sentence", "@sentence"),
]

# display all circles
color_dict=("#000000", "#FF0000", "#0000FF", "#00FF00")

p.circle(x="x",
         y="y",
         radius=.4,
         selection_color="black",
         fill_alpha=.5,
         line_alpha=.6,
         source=source)

p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p)

In [ ]:
plot_dict = AttributeDict(
    x = vec_tsne_bert[:,0],
    y = vec_tsne_bert[:,1],
    sentence = np.array(vocabulary),
    class_name_word = np.array(class_name_word)

)
source = ColumnDataSource(plot_dict)

TITLE = "Plot of BERT trained vectors in 2D using t-SNE"
TOOLS = "hover,pan,wheel_zoom,box_zoom,tap,reset,save"
p = figure(tools=TOOLS,
           title=TITLE,
           toolbar_location="above",
           plot_width=800,
           plot_height=800)
p.xaxis.axis_label = "t-SNE variable 1"
p.yaxis.axis_label = "t-SNE variable 2"

curdoc().theme = "light_minimal"
output_notebook()
#output_file("doc2vec.html")

# Show info per circle (pan)
p.hover.tooltips = [
    ("sentence", "@sentence"),
]

# display all circles
color_dict=("#000000", "#FF0000", "#0000FF", "#00FF00")

p.circle(x="x",
         y="y",
         radius=.4,
         selection_color="black",
         fill_alpha=.5,
         line_alpha=.6,
         source=source)

p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p)

In [ ]:
docvecs = doc2vec_model.docvecs
print(docvecs[0])
print(doc2vec_model.infer_vector(dataset.train.doc.values[0].words))

In [ ]:
vec_tsne = TSNE(n_components=2).fit_transform(doc2vec_model.docvecs)

color_word = dataset.train

plot_dict = AttributeDict(
    x = vec_tsne[:,0],
    y = vec_tsne[:,1],
    fill_color = np.array(color_word),
    legend_label = np.array(class_name_word),
    sentence = np.array(dataset.train.sentence.values),
    class_name_word = np.array(dataset.train["class"].values)

)
source = ColumnDataSource(plot_dict)

TITLE = "Plot of Doc2Vec sentence trained vectors in 2D using t-SNE"
TOOLS = "hover,pan,wheel_zoom,box_zoom,tap,reset,save"
p = figure(tools=TOOLS,
           title=TITLE,
           toolbar_location="above",
           plot_width=800,
           plot_height=800)
p.xaxis.axis_label = "t-SNE variable 1"
p.yaxis.axis_label = "t-SNE variable 2"
output_notebook()

curdoc().theme = "light_minimal"
#output_file("doc2vec.html")

# Show info per circle (pan)
p.hover.tooltips = [
    ("sentence", "@sentence"),
    ("class","@class_name_word")
]

# display all circles
color_dict=("#000000", "#FF0000", "#0000FF", "#00FF00")

p.circle(x="x",
         y="y",
         radius=.4,
         selection_color="black",
         fill_alpha=.5,
         line_alpha=.6,
         fill_color="fill_color",
         legend_group='class_name_word',
         source=source)

p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p)

# BERT model

In [ ]:
from sentence_transformers import SentenceTransformer

embedder = SentenceTransformer('roberta-base-nli-stsb-mean-tokens')

encoded_sentences = embedder.encode(dataset.train.sentence.values)

## Get the t-SNE of the BERT model

In [ ]:
vec_tsne_bert = TSNE(n_components=2).fit_transform(encoded_sentences)
vocabulary = dataset.train.sentence.values

In [ ]:
plot_dict = AttributeDict(
    x = vec_tsne_bert[:,0],
    y = vec_tsne_bert[:,1],
    sentence = np.array(vocabulary),
    class_name_word = np.array(class_name_word)

)
source = ColumnDataSource(plot_dict)

TITLE = "Plot of BERT trained vectors in 2D using t-SNE"
TOOLS = "hover,pan,wheel_zoom,box_zoom,tap,reset,save"
p = figure(tools=TOOLS,
           title=TITLE,
           toolbar_location="above",
           plot_width=800,
           plot_height=800)
p.xaxis.axis_label = "t-SNE variable 1"
p.yaxis.axis_label = "t-SNE variable 2"

curdoc().theme = "light_minimal"
output_notebook()
#output_file("doc2vec.html")

# Show info per circle (pan)
p.hover.tooltips = [
    ("sentence", "@sentence"),
]

# display all circles
color_dict=("#000000", "#FF0000", "#0000FF", "#00FF00")

p.circle(x="x",
         y="y",
         radius=.4,
         selection_color="black",
         fill_alpha=.5,
         line_alpha=.6,
         source=source)

p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p)

In [ ]:
plot_dict = AttributeDict(
    x = vec_tsne_bert[:,0],
    y = vec_tsne_bert[:,1],
    sentence = np.array(vocabulary),
    class_name_word = np.array(class_name_word)

)
source = ColumnDataSource(plot_dict)

TITLE = "Plot of BERT trained vectors in 2D using t-SNE"
TOOLS = "hover,pan,wheel_zoom,box_zoom,tap,reset,save"
p = figure(tools=TOOLS,
           title=TITLE,
           toolbar_location="above",
           plot_width=800,
           plot_height=800)
p.xaxis.axis_label = "t-SNE variable 1"
p.yaxis.axis_label = "t-SNE variable 2"

curdoc().theme = "light_minimal"
output_notebook()
#output_file("doc2vec.html")

# Show info per circle (pan)
p.hover.tooltips = [
    ("sentence", "@sentence"),
]

# display all circles
color_dict=("#000000", "#FF0000", "#0000FF", "#00FF00")

p.circle(x="x",
         y="y",
         radius=.4,
         selection_color="black",
         fill_alpha=.5,
         line_alpha=.6,
         source=source)

p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p)

In [ ]:
plot_dict = AttributeDict(
    x = vec_tsne_bert[:,0],
    y = vec_tsne_bert[:,1],
    sentence = np.array(vocabulary),
    class_name_word = np.array(class_name_word)

)
source = ColumnDataSource(plot_dict)

TITLE = "Plot of BERT trained vectors in 2D using t-SNE"
TOOLS = "hover,pan,wheel_zoom,box_zoom,tap,reset,save"
p = figure(tools=TOOLS,
           title=TITLE,
           toolbar_location="above",
           plot_width=800,
           plot_height=800)
p.xaxis.axis_label = "t-SNE variable 1"
p.yaxis.axis_label = "t-SNE variable 2"

curdoc().theme = "light_minimal"
output_notebook()
#output_file("doc2vec.html")

# Show info per circle (pan)
p.hover.tooltips = [
    ("sentence", "@sentence"),
]

# display all circles
color_dict=("#000000", "#FF0000", "#0000FF", "#00FF00")

p.circle(x="x",
         y="y",
         radius=.4,
         selection_color="black",
         fill_alpha=.5,
         line_alpha=.6,
         source=source)

p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p)

plot_dict = AttributeDict(
    x = vec_tsne_bert[:,0],
    y = vec_tsne_bert[:,1],
    sentence = np.array(vocabulary),
    class_name_word = np.array(class_name_word)

)
source = ColumnDataSource(plot_dict)

TITLE = "Plot of BERT trained vectors in 2D using t-SNE"
TOOLS = "hover,pan,wheel_zoom,box_zoom,tap,reset,save"
p = figure(tools=TOOLS,
           title=TITLE,
           toolbar_location="above",
           plot_width=800,
           plot_height=800)
p.xaxis.axis_label = "t-SNE variable 1"
p.yaxis.axis_label = "t-SNE variable 2"

curdoc().theme = "light_minimal"
output_notebook()
#output_file("doc2vec.html")

# Show info per circle (pan)
p.hover.tooltips = [
    ("sentence", "@sentence"),
]

# display all circles
color_dict=("#000000", "#FF0000", "#0000FF", "#00FF00")

p.circle(x="x",
         y="y",
         radius=.4,
         selection_color="black",
         fill_alpha=.5,
         line_alpha=.6,
         source=source)

p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p)

In [ ]:
docvecs = doc2vec_model.docvecs
print(docvecs[0])
print(doc2vec_model.infer_vector(dataset.train.doc.values[0].words))

In [ ]:
vec_tsne = TSNE(n_components=2).fit_transform(doc2vec_model.docvecs)

color_word = dataset.train

plot_dict = AttributeDict(
    x = vec_tsne[:,0],
    y = vec_tsne[:,1],
    fill_color = np.array(color_word),
    legend_label = np.array(class_name_word),
    sentence = np.array(dataset.train.sentence.values),
    class_name_word = np.array(dataset.train["class"].values)

)
source = ColumnDataSource(plot_dict)

TITLE = "Plot of Doc2Vec sentence trained vectors in 2D using t-SNE"
TOOLS = "hover,pan,wheel_zoom,box_zoom,tap,reset,save"
p = figure(tools=TOOLS,
           title=TITLE,
           toolbar_location="above",
           plot_width=800,
           plot_height=800)
p.xaxis.axis_label = "t-SNE variable 1"
p.yaxis.axis_label = "t-SNE variable 2"
output_notebook()

curdoc().theme = "light_minimal"
#output_file("doc2vec.html")

# Show info per circle (pan)
p.hover.tooltips = [
    ("sentence", "@sentence"),
    ("class","@class_name_word")
]

# display all circles
color_dict=("#000000", "#FF0000", "#0000FF", "#00FF00")

p.circle(x="x",
         y="y",
         radius=.4,
         selection_color="black",
         fill_alpha=.5,
         line_alpha=.6,
         fill_color="fill_color",
         legend_group='class_name_word',
         source=source)

p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p)

# BERT model

In [ ]:
from sentence_transformers import SentenceTransformer

embedder = SentenceTransformer('roberta-base-nli-stsb-mean-tokens')

encoded_sentences = embedder.encode(dataset.train.sentence.values)

## Get the t-SNE of the BERT model

In [ ]:
vec_tsne_bert = TSNE(n_components=2).fit_transform(encoded_sentences)
vocabulary = dataset.train.sentence.values

In [ ]:
plot_dict = AttributeDict(
    x = vec_tsne_bert[:,0],
    y = vec_tsne_bert[:,1],
    sentence = np.array(vocabulary),
    class_name_word = np.array(class_name_word)

)
source = ColumnDataSource(plot_dict)

TITLE = "Plot of BERT trained vectors in 2D using t-SNE"
TOOLS = "hover,pan,wheel_zoom,box_zoom,tap,reset,save"
p = figure(tools=TOOLS,
           title=TITLE,
           toolbar_location="above",
           plot_width=800,
           plot_height=800)
p.xaxis.axis_label = "t-SNE variable 1"
p.yaxis.axis_label = "t-SNE variable 2"

curdoc().theme = "light_minimal"
output_notebook()
#output_file("doc2vec.html")

# Show info per circle (pan)
p.hover.tooltips = [
    ("sentence", "@sentence"),
]

# display all circles
color_dict=("#000000", "#FF0000", "#0000FF", "#00FF00")

p.circle(x="x",
         y="y",
         radius=.4,
         selection_color="black",
         fill_alpha=.5,
         line_alpha=.6,
         source=source)

p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p)

In [ ]:
plot_dict = AttributeDict(
    x = vec_tsne_bert[:,0],
    y = vec_tsne_bert[:,1],
    sentence = np.array(vocabulary),
    class_name_word = np.array(class_name_word)

)
source = ColumnDataSource(plot_dict)

TITLE = "Plot of BERT trained vectors in 2D using t-SNE"
TOOLS = "hover,pan,wheel_zoom,box_zoom,tap,reset,save"
p = figure(tools=TOOLS,
           title=TITLE,
           toolbar_location="above",
           plot_width=800,
           plot_height=800)
p.xaxis.axis_label = "t-SNE variable 1"
p.yaxis.axis_label = "t-SNE variable 2"

curdoc().theme = "light_minimal"
output_notebook()
#output_file("doc2vec.html")

# Show info per circle (pan)
p.hover.tooltips = [
    ("sentence", "@sentence"),
]

# display all circles
color_dict=("#000000", "#FF0000", "#0000FF", "#00FF00")

p.circle(x="x",
         y="y",
         radius=.4,
         selection_color="black",
         fill_alpha=.5,
         line_alpha=.6,
         source=source)

p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p)

In [ ]:
plot_dict = AttributeDict(
    x = vec_tsne_bert[:,0],
    y = vec_tsne_bert[:,1],
    sentence = np.array(vocabulary),
    class_name_word = np.array(class_name_word)

)
source = ColumnDataSource(plot_dict)

TITLE = "Plot of BERT trained vectors in 2D using t-SNE"
TOOLS = "hover,pan,wheel_zoom,box_zoom,tap,reset,save"
p = figure(tools=TOOLS,
           title=TITLE,
           toolbar_location="above",
           plot_width=800,
           plot_height=800)
p.xaxis.axis_label = "t-SNE variable 1"
p.yaxis.axis_label = "t-SNE variable 2"

curdoc().theme = "light_minimal"
output_notebook()
#output_file("doc2vec.html")

# Show info per circle (pan)
p.hover.tooltips = [
    ("sentence", "@sentence"),
]

# display all circles
color_dict=("#000000", "#FF0000", "#0000FF", "#00FF00")

p.circle(x="x",
         y="y",
         radius=.4,
         selection_color="black",
         fill_alpha=.5,
         line_alpha=.6,
         source=source)

p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p)

plot_dict = AttributeDict(
    x = vec_tsne_bert[:,0],
    y = vec_tsne_bert[:,1],
    sentence = np.array(vocabulary),
    class_name_word = np.array(class_name_word)

)
source = ColumnDataSource(plot_dict)

TITLE = "Plot of BERT trained vectors in 2D using t-SNE"
TOOLS = "hover,pan,wheel_zoom,box_zoom,tap,reset,save"
p = figure(tools=TOOLS,
           title=TITLE,
           toolbar_location="above",
           plot_width=800,
           plot_height=800)
p.xaxis.axis_label = "t-SNE variable 1"
p.yaxis.axis_label = "t-SNE variable 2"

curdoc().theme = "light_minimal"
output_notebook()
#output_file("doc2vec.html")

# Show info per circle (pan)
p.hover.tooltips = [
    ("sentence", "@sentence"),
]

# display all circles
color_dict=("#000000", "#FF0000", "#0000FF", "#00FF00")

p.circle(x="x",
         y="y",
         radius=.4,
         selection_color="black",
         fill_alpha=.5,
         line_alpha=.6,
         source=source)

p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p)

In [ ]:
docvecs = doc2vec_model.docvecs
print(docvecs[0])
print(doc2vec_model.infer_vector(dataset.train.doc.values[0].words))

In [ ]:
vec_tsne = TSNE(n_components=2).fit_transform(doc2vec_model.docvecs)

color_word = dataset.train

plot_dict = AttributeDict(
    x = vec_tsne[:,0],
    y = vec_tsne[:,1],
    fill_color = np.array(color_word),
    legend_label = np.array(class_name_word),
    sentence = np.array(dataset.train.sentence.values),
    class_name_word = np.array(dataset.train["class"].values)

)
source = ColumnDataSource(plot_dict)

TITLE = "Plot of Doc2Vec sentence trained vectors in 2D using t-SNE"
TOOLS = "hover,pan,wheel_zoom,box_zoom,tap,reset,save"
p = figure(tools=TOOLS,
           title=TITLE,
           toolbar_location="above",
           plot_width=800,
           plot_height=800)
p.xaxis.axis_label = "t-SNE variable 1"
p.yaxis.axis_label = "t-SNE variable 2"
output_notebook()

curdoc().theme = "light_minimal"
#output_file("doc2vec.html")

# Show info per circle (pan)
p.hover.tooltips = [
    ("sentence", "@sentence"),
    ("class","@class_name_word")
]

# display all circles
color_dict=("#000000", "#FF0000", "#0000FF", "#00FF00")

p.circle(x="x",
         y="y",
         radius=.4,
         selection_color="black",
         fill_alpha=.5,
         line_alpha=.6,
         fill_color="fill_color",
         legend_group='class_name_word',
         source=source)

p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p)

# BERT model

In [ ]:
from sentence_transformers import SentenceTransformer

embedder = SentenceTransformer('roberta-base-nli-stsb-mean-tokens')

encoded_sentences = embedder.encode(dataset.train.sentence.values)

## Get the t-SNE of the BERT model

In [ ]:
vec_tsne_bert = TSNE(n_components=2).fit_transform(encoded_sentences)
vocabulary = dataset.train.sentence.values

In [ ]:
plot_dict = AttributeDict(
    x = vec_tsne_bert[:,0],
    y = vec_tsne_bert[:,1],
    sentence = np.array(vocabulary),
    class_name_word = np.array(class_name_word)

)
source = ColumnDataSource(plot_dict)

TITLE = "Plot of BERT trained vectors in 2D using t-SNE"
TOOLS = "hover,pan,wheel_zoom,box_zoom,tap,reset,save"
p = figure(tools=TOOLS,
           title=TITLE,
           toolbar_location="above",
           plot_width=800,
           plot_height=800)
p.xaxis.axis_label = "t-SNE variable 1"
p.yaxis.axis_label = "t-SNE variable 2"

curdoc().theme = "light_minimal"
output_notebook()
#output_file("doc2vec.html")

# Show info per circle (pan)
p.hover.tooltips = [
    ("sentence", "@sentence"),
]

# display all circles
color_dict=("#000000", "#FF0000", "#0000FF", "#00FF00")

p.circle(x="x",
         y="y",
         radius=.4,
         selection_color="black",
         fill_alpha=.5,
         line_alpha=.6,
         source=source)

p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p)

In [ ]:
plot_dict = AttributeDict(
    x = vec_tsne_bert[:,0],
    y = vec_tsne_bert[:,1],
    sentence = np.array(vocabulary),
    class_name_word = np.array(class_name_word)

)
source = ColumnDataSource(plot_dict)

TITLE = "Plot of BERT trained vectors in 2D using t-SNE"
TOOLS = "hover,pan,wheel_zoom,box_zoom,tap,reset,save"
p = figure(tools=TOOLS,
           title=TITLE,
           toolbar_location="above",
           plot_width=800,
           plot_height=800)
p.xaxis.axis_label = "t-SNE variable 1"
p.yaxis.axis_label = "t-SNE variable 2"

curdoc().theme = "light_minimal"
output_notebook()
#output_file("doc2vec.html")

# Show info per circle (pan)
p.hover.tooltips = [
    ("sentence", "@sentence"),
]

# display all circles
color_dict=("#000000", "#FF0000", "#0000FF", "#00FF00")

p.circle(x="x",
         y="y",
         radius=.4,
         selection_color="black",
         fill_alpha=.5,
         line_alpha=.6,
         source=source)

p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p)

In [ ]:
plot_dict = AttributeDict(
    x = vec_tsne_bert[:,0],
    y = vec_tsne_bert[:,1],
    sentence = np.array(vocabulary),
    class_name_word = np.array(class_name_word)

)
source = ColumnDataSource(plot_dict)

TITLE = "Plot of BERT trained vectors in 2D using t-SNE"
TOOLS = "hover,pan,wheel_zoom,box_zoom,tap,reset,save"
p = figure(tools=TOOLS,
           title=TITLE,
           toolbar_location="above",
           plot_width=800,
           plot_height=800)
p.xaxis.axis_label = "t-SNE variable 1"
p.yaxis.axis_label = "t-SNE variable 2"

curdoc().theme = "light_minimal"
output_notebook()
#output_file("doc2vec.html")

# Show info per circle (pan)
p.hover.tooltips = [
    ("sentence", "@sentence"),
]

# display all circles
color_dict=("#000000", "#FF0000", "#0000FF", "#00FF00")

p.circle(x="x",
         y="y",
         radius=.4,
         selection_color="black",
         fill_alpha=.5,
         line_alpha=.6,
         source=source)

p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p)

plot_dict = AttributeDict(
    x = vec_tsne_bert[:,0],
    y = vec_tsne_bert[:,1],
    sentence = np.array(vocabulary),
    class_name_word = np.array(class_name_word)

)
source = ColumnDataSource(plot_dict)

TITLE = "Plot of BERT trained vectors in 2D using t-SNE"
TOOLS = "hover,pan,wheel_zoom,box_zoom,tap,reset,save"
p = figure(tools=TOOLS,
           title=TITLE,
           toolbar_location="above",
           plot_width=800,
           plot_height=800)
p.xaxis.axis_label = "t-SNE variable 1"
p.yaxis.axis_label = "t-SNE variable 2"

curdoc().theme = "light_minimal"
output_notebook()
#output_file("doc2vec.html")

# Show info per circle (pan)
p.hover.tooltips = [
    ("sentence", "@sentence"),
]

# display all circles
color_dict=("#000000", "#FF0000", "#0000FF", "#00FF00")

p.circle(x="x",
         y="y",
         radius=.4,
         selection_color="black",
         fill_alpha=.5,
         line_alpha=.6,
         source=source)

p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p)

In [ ]:
docvecs = doc2vec_model.docvecs
print(docvecs[0])
print(doc2vec_model.infer_vector(dataset.train.doc.values[0].words))

In [ ]:
vec_tsne = TSNE(n_components=2).fit_transform(doc2vec_model.docvecs)

color_word = dataset.train

plot_dict = AttributeDict(
    x = vec_tsne[:,0],
    y = vec_tsne[:,1],
    fill_color = np.array(color_word),
    legend_label = np.array(class_name_word),
    sentence = np.array(dataset.train.sentence.values),
    class_name_word = np.array(dataset.train["class"].values)

)
source = ColumnDataSource(plot_dict)

TITLE = "Plot of Doc2Vec sentence trained vectors in 2D using t-SNE"
TOOLS = "hover,pan,wheel_zoom,box_zoom,tap,reset,save"
p = figure(tools=TOOLS,
           title=TITLE,
           toolbar_location="above",
           plot_width=800,
           plot_height=800)
p.xaxis.axis_label = "t-SNE variable 1"
p.yaxis.axis_label = "t-SNE variable 2"
output_notebook()

curdoc().theme = "light_minimal"
#output_file("doc2vec.html")

# Show info per circle (pan)
p.hover.tooltips = [
    ("sentence", "@sentence"),
    ("class","@class_name_word")
]

# display all circles
color_dict=("#000000", "#FF0000", "#0000FF", "#00FF00")

p.circle(x="x",
         y="y",
         radius=.4,
         selection_color="black",
         fill_alpha=.5,
         line_alpha=.6,
         fill_color="fill_color",
         legend_group='class_name_word',
         source=source)

p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p)

# BERT model

In [ ]:
from sentence_transformers import SentenceTransformer

embedder = SentenceTransformer('roberta-base-nli-stsb-mean-tokens')

encoded_sentences = embedder.encode(dataset.train.sentence.values)

## Get the t-SNE of the BERT model

In [ ]:
vec_tsne_bert = TSNE(n_components=2).fit_transform(encoded_sentences)
vocabulary = dataset.train.sentence.values

In [ ]:
plot_dict = AttributeDict(
    x = vec_tsne_bert[:,0],
    y = vec_tsne_bert[:,1],
    sentence = np.array(vocabulary),
    class_name_word = np.array(class_name_word)

)
source = ColumnDataSource(plot_dict)

TITLE = "Plot of BERT trained vectors in 2D using t-SNE"
TOOLS = "hover,pan,wheel_zoom,box_zoom,tap,reset,save"
p = figure(tools=TOOLS,
           title=TITLE,
           toolbar_location="above",
           plot_width=800,
           plot_height=800)
p.xaxis.axis_label = "t-SNE variable 1"
p.yaxis.axis_label = "t-SNE variable 2"

curdoc().theme = "light_minimal"
output_notebook()
#output_file("doc2vec.html")

# Show info per circle (pan)
p.hover.tooltips = [
    ("sentence", "@sentence"),
]

# display all circles
color_dict=("#000000", "#FF0000", "#0000FF", "#00FF00")

p.circle(x="x",
         y="y",
         radius=.4,
         selection_color="black",
         fill_alpha=.5,
         line_alpha=.6,
         source=source)

p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p)

In [ ]:
plot_dict = AttributeDict(
    x = vec_tsne_bert[:,0],
    y = vec_tsne_bert[:,1],
    sentence = np.array(vocabulary),
    class_name_word = np.array(class_name_word)

)
source = ColumnDataSource(plot_dict)

TITLE = "Plot of BERT trained vectors in 2D using t-SNE"
TOOLS = "hover,pan,wheel_zoom,box_zoom,tap,reset,save"
p = figure(tools=TOOLS,
           title=TITLE,
           toolbar_location="above",
           plot_width=800,
           plot_height=800)
p.xaxis.axis_label = "t-SNE variable 1"
p.yaxis.axis_label = "t-SNE variable 2"

curdoc().theme = "light_minimal"
output_notebook()
#output_file("doc2vec.html")

# Show info per circle (pan)
p.hover.tooltips = [
    ("sentence", "@sentence"),
]

# display all circles
color_dict=("#000000", "#FF0000", "#0000FF", "#00FF00")

p.circle(x="x",
         y="y",
         radius=.4,
         selection_color="black",
         fill_alpha=.5,
         line_alpha=.6,
         source=source)

p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p)

In [ ]:
plot_dict = AttributeDict(
    x = vec_tsne_bert[:,0],
    y = vec_tsne_bert[:,1],
    sentence = np.array(vocabulary),
    class_name_word = np.array(class_name_word)

)
source = ColumnDataSource(plot_dict)

TITLE = "Plot of BERT trained vectors in 2D using t-SNE"
TOOLS = "hover,pan,wheel_zoom,box_zoom,tap,reset,save"
p = figure(tools=TOOLS,
           title=TITLE,
           toolbar_location="above",
           plot_width=800,
           plot_height=800)
p.xaxis.axis_label = "t-SNE variable 1"
p.yaxis.axis_label = "t-SNE variable 2"

curdoc().theme = "light_minimal"
output_notebook()
#output_file("doc2vec.html")

# Show info per circle (pan)
p.hover.tooltips = [
    ("sentence", "@sentence"),
]

# display all circles
color_dict=("#000000", "#FF0000", "#0000FF", "#00FF00")

p.circle(x="x",
         y="y",
         radius=.4,
         selection_color="black",
         fill_alpha=.5,
         line_alpha=.6,
         source=source)

p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p)

plot_dict = AttributeDict(
    x = vec_tsne_bert[:,0],
    y = vec_tsne_bert[:,1],
    sentence = np.array(vocabulary),
    class_name_word = np.array(class_name_word)

)
source = ColumnDataSource(plot_dict)

TITLE = "Plot of BERT trained vectors in 2D using t-SNE"
TOOLS = "hover,pan,wheel_zoom,box_zoom,tap,reset,save"
p = figure(tools=TOOLS,
           title=TITLE,
           toolbar_location="above",
           plot_width=800,
           plot_height=800)
p.xaxis.axis_label = "t-SNE variable 1"
p.yaxis.axis_label = "t-SNE variable 2"

curdoc().theme = "light_minimal"
output_notebook()
#output_file("doc2vec.html")

# Show info per circle (pan)
p.hover.tooltips = [
    ("sentence", "@sentence"),
]

# display all circles
color_dict=("#000000", "#FF0000", "#0000FF", "#00FF00")

p.circle(x="x",
         y="y",
         radius=.4,
         selection_color="black",
         fill_alpha=.5,
         line_alpha=.6,
         source=source)

p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p)

In [ ]:
docvecs = doc2vec_model.docvecs
print(docvecs[0])
print(doc2vec_model.infer_vector(dataset.train.doc.values[0].words))

In [ ]:
vec_tsne = TSNE(n_components=2).fit_transform(doc2vec_model.docvecs)

color_word = dataset.train

plot_dict = AttributeDict(
    x = vec_tsne[:,0],
    y = vec_tsne[:,1],
    fill_color = np.array(color_word),
    legend_label = np.array(class_name_word),
    sentence = np.array(dataset.train.sentence.values),
    class_name_word = np.array(dataset.train["class"].values)

)
source = ColumnDataSource(plot_dict)

TITLE = "Plot of Doc2Vec sentence trained vectors in 2D using t-SNE"
TOOLS = "hover,pan,wheel_zoom,box_zoom,tap,reset,save"
p = figure(tools=TOOLS,
           title=TITLE,
           toolbar_location="above",
           plot_width=800,
           plot_height=800)
p.xaxis.axis_label = "t-SNE variable 1"
p.yaxis.axis_label = "t-SNE variable 2"
output_notebook()

curdoc().theme = "light_minimal"
#output_file("doc2vec.html")

# Show info per circle (pan)
p.hover.tooltips = [
    ("sentence", "@sentence"),
    ("class","@class_name_word")
]

# display all circles
color_dict=("#000000", "#FF0000", "#0000FF", "#00FF00")

p.circle(x="x",
         y="y",
         radius=.4,
         selection_color="black",
         fill_alpha=.5,
         line_alpha=.6,
         fill_color="fill_color",
         legend_group='class_name_word',
         source=source)

p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p)

# BERT model

In [ ]:
from sentence_transformers import SentenceTransformer

embedder = SentenceTransformer('roberta-base-nli-stsb-mean-tokens')

encoded_sentences = embedder.encode(dataset.train.sentence.values)

## Get the t-SNE of the BERT model

In [ ]:
vec_tsne_bert = TSNE(n_components=2).fit_transform(encoded_sentences)
vocabulary = dataset.train.sentence.values

In [ ]:
plot_dict = AttributeDict(
    x = vec_tsne_bert[:,0],
    y = vec_tsne_bert[:,1],
    sentence = np.array(vocabulary),
    class_name_word = np.array(class_name_word)

)
source = ColumnDataSource(plot_dict)

TITLE = "Plot of BERT trained vectors in 2D using t-SNE"
TOOLS = "hover,pan,wheel_zoom,box_zoom,tap,reset,save"
p = figure(tools=TOOLS,
           title=TITLE,
           toolbar_location="above",
           plot_width=800,
           plot_height=800)
p.xaxis.axis_label = "t-SNE variable 1"
p.yaxis.axis_label = "t-SNE variable 2"

curdoc().theme = "light_minimal"
output_notebook()
#output_file("doc2vec.html")

# Show info per circle (pan)
p.hover.tooltips = [
    ("sentence", "@sentence"),
]

# display all circles
color_dict=("#000000", "#FF0000", "#0000FF", "#00FF00")

p.circle(x="x",
         y="y",
         radius=.4,
         selection_color="black",
         fill_alpha=.5,
         line_alpha=.6,
         source=source)

p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p)

In [ ]:
plot_dict = AttributeDict(
    x = vec_tsne_bert[:,0],
    y = vec_tsne_bert[:,1],
    sentence = np.array(vocabulary),
    class_name_word = np.array(class_name_word)

)
source = ColumnDataSource(plot_dict)

TITLE = "Plot of BERT trained vectors in 2D using t-SNE"
TOOLS = "hover,pan,wheel_zoom,box_zoom,tap,reset,save"
p = figure(tools=TOOLS,
           title=TITLE,
           toolbar_location="above",
           plot_width=800,
           plot_height=800)
p.xaxis.axis_label = "t-SNE variable 1"
p.yaxis.axis_label = "t-SNE variable 2"

curdoc().theme = "light_minimal"
output_notebook()
#output_file("doc2vec.html")

# Show info per circle (pan)
p.hover.tooltips = [
    ("sentence", "@sentence"),
]

# display all circles
color_dict=("#000000", "#FF0000", "#0000FF", "#00FF00")

p.circle(x="x",
         y="y",
         radius=.4,
         selection_color="black",
         fill_alpha=.5,
         line_alpha=.6,
         source=source)

p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p)

In [ ]:
plot_dict = AttributeDict(
    x = vec_tsne_bert[:,0],
    y = vec_tsne_bert[:,1],
    sentence = np.array(vocabulary),
    class_name_word = np.array(class_name_word)

)
source = ColumnDataSource(plot_dict)

TITLE = "Plot of BERT trained vectors in 2D using t-SNE"
TOOLS = "hover,pan,wheel_zoom,box_zoom,tap,reset,save"
p = figure(tools=TOOLS,
           title=TITLE,
           toolbar_location="above",
           plot_width=800,
           plot_height=800)
p.xaxis.axis_label = "t-SNE variable 1"
p.yaxis.axis_label = "t-SNE variable 2"

curdoc().theme = "light_minimal"
output_notebook()
#output_file("doc2vec.html")

# Show info per circle (pan)
p.hover.tooltips = [
    ("sentence", "@sentence"),
]

# display all circles
color_dict=("#000000", "#FF0000", "#0000FF", "#00FF00")

p.circle(x="x",
         y="y",
         radius=.4,
         selection_color="black",
         fill_alpha=.5,
         line_alpha=.6,
         source=source)

p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p)

plot_dict = AttributeDict(
    x = vec_tsne_bert[:,0],
    y = vec_tsne_bert[:,1],
    sentence = np.array(vocabulary),
    class_name_word = np.array(class_name_word)

)
source = ColumnDataSource(plot_dict)

TITLE = "Plot of BERT trained vectors in 2D using t-SNE"
TOOLS = "hover,pan,wheel_zoom,box_zoom,tap,reset,save"
p = figure(tools=TOOLS,
           title=TITLE,
           toolbar_location="above",
           plot_width=800,
           plot_height=800)
p.xaxis.axis_label = "t-SNE variable 1"
p.yaxis.axis_label = "t-SNE variable 2"

curdoc().theme = "light_minimal"
output_notebook()
#output_file("doc2vec.html")

# Show info per circle (pan)
p.hover.tooltips = [
    ("sentence", "@sentence"),
]

# display all circles
color_dict=("#000000", "#FF0000", "#0000FF", "#00FF00")

p.circle(x="x",
         y="y",
         radius=.4,
         selection_color="black",
         fill_alpha=.5,
         line_alpha=.6,
         source=source)

p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p)

In [39]:
docvecs = doc2vec_model.docvecs
print(docvecs[0])
print(doc2vec_model.infer_vector(dataset.train.doc.values[0].words))

[-2.5301938   0.8577112  -1.133557    0.41244704  0.29898098 -1.5561557
 -1.3114977  -1.0765412  -1.0237803  -1.0805854   0.7517884   1.3011371
  2.6683095   0.23335105 -1.9845157  -3.1121237  -0.77441806 -0.30762368
  1.0664773   0.70978934 -1.1972789  -1.8598366  -1.2793047  -0.38386145
 -0.12632996 -0.13723433 -0.05695308  0.26827598  0.49709436 -0.35507977
 -0.21036208 -0.4060113   1.0179808  -1.4772539  -1.029674   -1.0657368
 -1.1598835   0.22190884 -0.4017658  -1.1223317  -1.8644214   0.24103406
  0.09477831  0.890395    1.2215052   0.44075644 -0.7238971  -2.6096463
 -2.7384253   2.4185088  -2.3352869  -0.49660042  0.96990883 -1.1435165
 -0.3198669  -0.07223254  0.50009274  0.5545506  -0.40994018 -0.75209355
 -0.2961104  -0.25702167  1.5777469   0.7424367  -0.04122495 -0.10209477
  0.50789887 -0.88307506 -0.8880999   0.78167677 -0.74615973  0.01241894
  1.3692627   0.10308021 -0.24192078  0.6519565   1.048242    0.08129063
  0.9761686   1.9510846   1.8412081  -0.07635148  3.5468

In [ ]:
vec_tsne = TSNE(n_components=2).fit_transform(doc2vec_model.docvecs)

color_word = dataset.train

plot_dict = AttributeDict(
    x = vec_tsne[:,0],
    y = vec_tsne[:,1],
    fill_color = np.array(color_word),
    legend_label = np.array(class_name_word),
    sentence = np.array(dataset.train.sentence.values),
    class_name_word = np.array(dataset.train["class"].values)

)
source = ColumnDataSource(plot_dict)

TITLE = "Plot of Doc2Vec sentence trained vectors in 2D using t-SNE"
TOOLS = "hover,pan,wheel_zoom,box_zoom,tap,reset,save"
p = figure(tools=TOOLS,
           title=TITLE,
           toolbar_location="above",
           plot_width=800,
           plot_height=800)
p.xaxis.axis_label = "t-SNE variable 1"
p.yaxis.axis_label = "t-SNE variable 2"
output_notebook()

curdoc().theme = "light_minimal"
#output_file("doc2vec.html")

# Show info per circle (pan)
p.hover.tooltips = [
    ("sentence", "@sentence"),
    ("class","@class_name_word")
]

# display all circles
color_dict=("#000000", "#FF0000", "#0000FF", "#00FF00")

p.circle(x="x",
         y="y",
         radius=.4,
         selection_color="black",
         fill_alpha=.5,
         line_alpha=.6,
         fill_color="fill_color",
         legend_group='class_name_word',
         source=source)

p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p)

# BERT model

In [ ]:
from sentence_transformers import SentenceTransformer

embedder = SentenceTransformer('roberta-base-nli-stsb-mean-tokens')

encoded_sentences = embedder.encode(dataset.train.sentence.values)

## Get the t-SNE of the BERT model

In [ ]:
vec_tsne_bert = TSNE(n_components=2).fit_transform(encoded_sentences)
vocabulary = dataset.train.sentence.values

In [ ]:
plot_dict = AttributeDict(
    x = vec_tsne_bert[:,0],
    y = vec_tsne_bert[:,1],
    sentence = np.array(vocabulary),
    class_name_word = np.array(class_name_word)

)
source = ColumnDataSource(plot_dict)

TITLE = "Plot of BERT trained vectors in 2D using t-SNE"
TOOLS = "hover,pan,wheel_zoom,box_zoom,tap,reset,save"
p = figure(tools=TOOLS,
           title=TITLE,
           toolbar_location="above",
           plot_width=800,
           plot_height=800)
p.xaxis.axis_label = "t-SNE variable 1"
p.yaxis.axis_label = "t-SNE variable 2"

curdoc().theme = "light_minimal"
output_notebook()
#output_file("doc2vec.html")

# Show info per circle (pan)
p.hover.tooltips = [
    ("sentence", "@sentence"),
]

# display all circles
color_dict=("#000000", "#FF0000", "#0000FF", "#00FF00")

p.circle(x="x",
         y="y",
         radius=.4,
         selection_color="black",
         fill_alpha=.5,
         line_alpha=.6,
         source=source)

p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p)

In [ ]:
plot_dict = AttributeDict(
    x = vec_tsne_bert[:,0],
    y = vec_tsne_bert[:,1],
    sentence = np.array(vocabulary),
    class_name_word = np.array(class_name_word)

)
source = ColumnDataSource(plot_dict)

TITLE = "Plot of BERT trained vectors in 2D using t-SNE"
TOOLS = "hover,pan,wheel_zoom,box_zoom,tap,reset,save"
p = figure(tools=TOOLS,
           title=TITLE,
           toolbar_location="above",
           plot_width=800,
           plot_height=800)
p.xaxis.axis_label = "t-SNE variable 1"
p.yaxis.axis_label = "t-SNE variable 2"

curdoc().theme = "light_minimal"
output_notebook()
#output_file("doc2vec.html")

# Show info per circle (pan)
p.hover.tooltips = [
    ("sentence", "@sentence"),
]

# display all circles
color_dict=("#000000", "#FF0000", "#0000FF", "#00FF00")

p.circle(x="x",
         y="y",
         radius=.4,
         selection_color="black",
         fill_alpha=.5,
         line_alpha=.6,
         source=source)

p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p)

In [ ]:
plot_dict = AttributeDict(
    x = vec_tsne_bert[:,0],
    y = vec_tsne_bert[:,1],
    sentence = np.array(vocabulary),
    class_name_word = np.array(class_name_word)

)
source = ColumnDataSource(plot_dict)

TITLE = "Plot of BERT trained vectors in 2D using t-SNE"
TOOLS = "hover,pan,wheel_zoom,box_zoom,tap,reset,save"
p = figure(tools=TOOLS,
           title=TITLE,
           toolbar_location="above",
           plot_width=800,
           plot_height=800)
p.xaxis.axis_label = "t-SNE variable 1"
p.yaxis.axis_label = "t-SNE variable 2"

curdoc().theme = "light_minimal"
output_notebook()
#output_file("doc2vec.html")

# Show info per circle (pan)
p.hover.tooltips = [
    ("sentence", "@sentence"),
]

# display all circles
color_dict=("#000000", "#FF0000", "#0000FF", "#00FF00")

p.circle(x="x",
         y="y",
         radius=.4,
         selection_color="black",
         fill_alpha=.5,
         line_alpha=.6,
         source=source)

p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p)